### Fragment Annotation

In this project, we modify the souce code from the pLannotate, we create a `segment()` function in the `annotate.py`. In this python notebook, we mainly build the algorithm based on the exact match list that we have found in the `segment()`.

In [1]:
import biotite.sequence as bioseq
import biotite.sequence.graphics as graphics

# plot_annotate() the function to visualize the fragment annotation result. We used the biotite package in order to visualize the plasmid.
def plot_annotate(result, length, file, method):
	# greedy result
	features = []
	for data in result:
		print(data)
		features.append(bioseq.Feature(
				"CDS",
				[bioseq.Location(data[1],data[2])],
  			{"product": data[3]}
		))
	annotation = bioseq.Annotation(features)


	fig = plt.figure(figsize=(10, 10))
	ax = fig.add_subplot(111, projection="polar")
	graphics.plot_plasmid_map(
		ax, annotation, plasmid_size=length, label="Plasmid",label_properties={"fontsize": 8}
	)

	ticks = ax.get_xticks()
	labels = ax.get_xticklabels()

	plt.savefig('data/media/exp4/'+file+'_' + method + '.png')

In [2]:
# greedy_select() the function to select the fragment from the blast result. We used the greedy algorithm to select the fragment.

def greedy_select(hits, result, fragment_res, qstart, qend, tag):
	# find the max fragment inside the exact match list, also the max fragment should not has overlap with the qstart and qend
	longest = None
	while longest == None:
		start_i = hits['query_len'].idxmax()
		tmp_qstart = hits.loc[start_i]['qstart']
		tmp_qend = hits.loc[start_i]['qend']
		seqid = hits.loc[start_i]['sseqid']  
		if tmp_qend <= qend and tmp_qstart >= qstart:
			# has full overlap we don't want this
			hits = hits.drop(start_i)
			if len(hits.index) == 0:
				return hits, result, fragment_res, qstart, qend, False
			continue
		qstart = min(tmp_qstart,qstart)
		qend = max(tmp_qend,qend)
		longest = tmp_qend - tmp_qstart
		if longest < 150:
			hits = hits.drop(start_i)
			if len(hits.index) == 0:
				return hits, result, fragment_res, qstart, qend, False
			return hits, result, fragment_res, qstart, qend, True
		result.append((start_i, tmp_qstart, tmp_qend, seqid))
		fragment_res -= 1
		break
	while fragment_res >= 0:
		maxextralen = 0
		maxseq = None
		newqstart = qstart
		newqend = qend
		for i in hits.index:
			# find the overlap part first
			tmpstart = hits.loc[i]['qstart']
			tmpend = hits.loc[i]['qend']
			tmpseqid = hits.loc[i]['sseqid']
			if tmpend-tmpstart >= fragment_len:
				if tmpstart <= qend+41: # right overlap
					extralen = tmpend - qend
					if extralen > maxextralen:
						newqend = tmpend
						maxextralen = extralen
						maxseq = (i, tmpstart,tmpend,tmpseqid)
				elif tmpend >= qstart-41: # left overlap
					extralen = qstart - tmpstart
					if extralen > maxextralen:
						newqstart = tmpstart
						maxextralen = extralen
						maxseq = (i, tmpstart, tmpend, tmpseqid)
		if maxseq is None:
			break
		qstart = newqstart
		qend = newqend
		result.append(maxseq)
		hits.drop(maxseq[0])
		fragment_res -= 1
	return hits, result, fragment_res, qstart, qend, True

In [11]:
# Example code for the one-shot version experiment


from plannotate.annotate import segment
import os
from Bio import SeqIO
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

data_dir = 'data/plasmid/'
fragment_number = 10
fragment_len= 300
cost = []
for file in os.listdir(data_dir):
	# if file[:2] != 'SC':
	# 	continue
	if file[:4] != 'HIGH':
		continue
	print('Loading the data from: ', file)
	filename = data_dir + file
	output = []
	record = SeqIO.read(filename, format= 'snapgene')
	seq = str(record.seq)
	cache = [0] * len(seq)
	print('Finding the exact match... ')
	hits = segment(seq, is_detailed = True, linear= True, db='HIGHGC')
	columns = ['qstart', 'qend', 'sseqid','sstart', 'send']
	hits = hits[columns]
	hits['query_len'] = hits['qend'] - hits['qstart']
	fragment_res = fragment_number
	result = []
	qstart = len(cache)
	qend = 0
	tag = True
	# find the max and then find the maximum continuos sack
	while fragment_res > 0:
		hits, result, fragment_res, qstart, qend, tag = greedy_select(hits, result, fragment_res, qstart, qend, tag)
		if not tag:
			break
	c = get_cost(result, len(cache))
	cost.append({'id': file, 'total_len': len(cache), 'previous_cost': len(cache)*0.08, 'synthesis_len': c, 'cost': c*0.08})
# cost_df = pd.DataFrame(cost)
# cost_df.to_csv('data/media/exp4/HIGHGC.csv', index=False)
	plot_annotate(result, len(cache), file, 'one-shot-IGR')
		

In [5]:
# Get cost function

def get_cost(result, total_len):
  init = [0] * total_len
  for data in result:
    init[data[1]:data[2]+1] = [1] * (data[2]-data[1]+1)
  return total_len - sum(init)